In [94]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [95]:
# Load data from Excel files
train_df = pd.read_excel("train.xlsx")
test_df = pd.read_excel("test.xlsx")
# Rename columns for clarity

train_df.columns = ['id', 'user', 'tweet']
test_df.columns = ['id', 'user', 'tweet']

In [96]:
def reshape_data(df):
    # Sample a subset of data to reduce memory usage
    sample_size = min(500, len(df))  # initially 10000 / karnel crashed
    if sample_size < len(df):
        df_sample = df.sample(n=sample_size, replace=True, random_state=42)
    else:
        df_sample = df.copy()

    df1 = df_sample.copy()
    df2 = df_sample.copy()

    # Merge dataframes to create pairs
    merged_df = pd.merge(df1, df2, how='cross', suffixes=('_1', '_2'))

    # Filter out pairs where user1 equals user2 (same tweet)
    merged_df = merged_df[merged_df['user_1'] != merged_df['user_2']]

    # Create is_same column
    merged_df['is_same'] = merged_df['tweet_1'] == merged_df['tweet_2']

    return merged_df[['id_1', 'user_1', 'tweet_1', 'user_2', 'tweet_2', 'is_same']]

In [97]:
# Reshape train and test data
train_data = reshape_data(train_df)
test_data = reshape_data(test_df)

# Load pre-trained transformer model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModel.from_pretrained(model_name)

# Define model architecture
input_ids1 = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)
input_ids2 = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [98]:
# Encode tweet pairs using the transformer model
embedding1 = transformer_model(input_ids1)[0][:, 0, :]
embedding2 = transformer_model(input_ids2)[0][:, 0, :]

In [99]:
# Calculate Manhattan distance between tweet embeddings
distance = tf.keras.layers.Lambda(lambda x: tf.abs(x[0] - x[1]))([embedding1, embedding2])

In [100]:
# Tokenize tweet pairs
def tokenize_tweets(tweet_pairs):
    return tokenizer(tweet_pairs.values.tolist(), padding=True, truncation=True, return_tensors="tf")

train_tokenized_pairs = tokenize_tweets(train_data[['tweet_1', 'tweet_2']])
test_tokenized_pairs = tokenize_tweets(test_data[['tweet_1', 'tweet_2']])

In [101]:
# Define the model with inputs and outputs
dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')(distance)
model = tf.keras.Model(inputs=[input_ids1, input_ids2], outputs=dense_layer)


In [102]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [103]:

# Train model
model.fit([train_tokenized_pairs['input_ids'], train_tokenized_pairs['input_ids']], 
          train_data['is_same'], epochs=5, batch_size=32)


Epoch 1/5


  20/7204 [..............................] - ETA: 110:20:34 - loss: 0.5394 - accuracy: 0.7922

In [ ]:
model.save('K:\STUDIES\----- M1 AI\S2\NLP\NLP-LAB\PROJ 01')


In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate([test_tokenized_pairs['input_ids'][:, 0, :], test_tokenized_pairs['input_ids'][:, 1, :]],
                                          test_data['is_same'])
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")